# Exercise 9: Visualization for ML Interpretability

Here we are going to try using an out-of-the-box tool for some ML interpretability which supports some of the techniques we introduced in lecture.

The tool we will try out today is [interpretML](https://github.com/interpretml/interpret/), an interpretability Python module maintained by Microsoft. See this [blog post](https://towardsdatascience.com/interpretml-another-way-to-explain-your-model-b7faf0a384f8) for an overview of the kinds of things you can do with interpretML, including code snippets that will be helpful for this exercise.

## Preparing the dataset

For today's exercise, we'll be working with a dataset on emergency room visits and what factors predict whether a person is admitted or discharged. Before modeling this data, we need to do a bit of preprocessing. We provide this code since it's not the emphasis of our exercise.

We'll start by loading the dataset. The original is over 500,000 records, but we share a subsampled version with 60,000 records on the course website.

In [49]:
import pandas as pd

# students may need to change this path to whereever they downloaded the data
df = pd.read_csv("../data/hospital-admissions.csv") 
df.head()

,Unnamed: 0.1,Unnamed: 0,dep_name,esi,age,gender,ethnicity,race,lang,religion,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
0,280199,280200,A,2.0,48.0,Female,Hispanic or Latino,Other,English,Catholic,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,102688,102689,C,4.0,20.0,Female,Non-Hispanic,White or Caucasian,English,Catholic,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,143749,143750,B,2.0,75.0,Female,Non-Hispanic,Black or African American,English,Pentecostal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,375544,375545,B,4.0,50.0,Female,Hispanic or Latino,Other,English,Pentecostal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,312621,312622,A,4.0,26.0,Male,Non-Hispanic,Black or African American,English,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The next block of code handles some preprocessing, namely, relabeling strings in our dataset as integers. This is necessary for modeling the data, and the way we do it here preserves a connection between the original string values and their "dummy coded" integers.

In [29]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import column_or_1d

# set up for relabeling
class MyLabelEncoder(LabelEncoder):

    def fit(self, y):
        y = column_or_1d(y, warn=True)
        self.classes_ = pd.Series(y).unique()
        return self

def code_string(df, colname):
    le = MyLabelEncoder()
    arr = df[colname].values
    le = le.fit(arr)
    df[colname] = le.transform(arr)

    return (df, le)

RECODE_SET = ['disposition', 'dep_name', 'gender', 'ethnicity', 'race', 'lang', 'religion', 'maritalstatus', 'employstatus', 'insurance_status', 'arrivalmode', 'previousdispo']

In [30]:
# prepare dataset:
# eliminate timeseries variables
for var in ['arrivalmonth', 'arrivalday', 'arrivalhour_bin']:
    df.pop(var)
# TODO: normalize X values?
# make sure discharge maps to 0 when we use MyLabelEncoder
df.sort_values(by = 'disposition', ascending = True) 
# call label encoder to code strings as numbers for modeling
label_encoders = {}
for var in RECODE_SET:
    df, label_encoders[var] = code_string(df, var)
# separate features and target
y = df.pop('disposition').values
X = df

## Modeling

Let's do the typical modeling workflow. I want you to try since the syntax is concise, and this is something you'll probably need to do often as a data scientist.

We'll start by using scikit-learn's test-train split, which is described well in this [blog post](https://www.sharpsightlabs.com/blog/scikit-train_test_split/). For our case, we already have clean input data separated into features `X` and classification labels `y`. 

All you need to do is:

- Filter the features in `X` to only those that you would like to include in the model. You should pick 10 features to use as predictors in the model.
- Pass `X` and `y` to `sklearn.model_selection.test_train_split`
- Be sure to stratify on the outcome variable `y` so that the test and train sets have proportional numbers of people admitted to the emergency room

In [ ]:
# PROMPT: drop all but 10 features from X

In [32]:
# PROMPT: split X, y into non-overlapping subsamples for testing and training our model

*PROMPT: What columns of X did you keep to include as predictors in your model? Why did you want to include these in particular?*

Next, we will fit a model to our training data. Let's use a type of GAM called an Explainable Boosting Machine (EBM). This is basically a Generalized Additive Model fit using a variation on decision trees. We'll use the classifier version of this model, `interpret.glassbox.ExplainableBoostingClassifier`.

In [ ]:
# PROMPT: fit an EBM classifier to the training sample

You are going to see a warning about missing data. It's best not to ignore that in practice.

Let's compute our test set accuracy using an AUC (Area Under the receiver operating characteristic Curve), which is a common measure of accuracy for classifiers. To do this, you'll need to use `sklearn.metrics.accuracy_score` (see [API documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)).

In [ ]:
# PROMPT: compute test set accuracy as a percentage.
# HINT: you'll need to round the output of ebm.predict() to get binary predictions

## Interpretability!

Now that we have a model fit to our data, we can use the interpretability tools provided by `interpretML` to explain what our model has learned. You'll need `interpret.show` along with the `ebm.explain_global()` and `ebm.explain_local()` methods.

In [ ]:
# PROMPT: generate interpret's interactive widget for global explanations

*PROMPT: Look at the "Summary" of feature importance. What are the top three predictors? Do these make sense to you? Are they what you expected?*

*PROMPT: Look through the shape functions for the features you've chosen to include in the model. Describe one or two unexpected patterns learned by the model. Reflect a bit on whether interpreting these charts feels rigorous to you.*

In [ ]:
# PROMPT: generate interpret's interactive widget for local explanations

*PROMPT: Pick two cases in the test set to examine using the dropdown menu. These should be one case where the model performs well and one case where the model performs poorly. In each case, describe how the model makes its prediction, focusing on which features contribute very differently to the two predictions. Please note if anything about the explanation doesn't make sense to you such that it might undermine your trust in the model.*